# Import Libraries

In [2]:
import os
import shutil
import time
import json
import requests
import tempfile
import numpy as np
import pandas as pd
from zipfile import ZipFile

import boto3
import sagemaker

from datetime import datetime

# Setup Region, session, and role

In [3]:
region = os.environ["AWS_REGION"]
boto_session = boto3.Session(region_name=region)

account_id = boto_session.client("sts").get_caller_identity()["Account"]

# Get the AWS EventBridge client
event_bridge_client = boto3.client('events')
lambda_client = boto3.client('lambda')

sns_client = boto3.client('sns')
cloudwatch = boto3.client('cloudwatch')

iam_client = boto3.client('iam')

# Initialize variable

In [16]:
# S3 prefix
bucket = "artwork-content-trial-bucket"
prefix="mch-artwork-content"
train_data_dir_prefix="data"
pipeline_dir_prefix="pipeline-data"
# training_input_prefix = "content_ovr_anonymised.csv"
training_input_prefix = "ovr_content_data"

model_alarm_name='Model_Version_Registration_Alarm'
pipeline_alarm_name='SageMaker_Pipeline_Execution_Alarm'
pipeline_name='artwork-content-pipeline-demo'
sns_topic_name = 'PipelineExecutionAlarmTopic'
sns_topic_arn = f'arn:aws:sns:{region}:{account_id}:{sns_topic_name}'
lambda_function_name="sns-lambda-evt-trigger-fcn"
lambda_arn="arn:aws:lambda:us-east-1:791574662255:function:sns-lambda-evt-trigger-fcn"

# =========================================

# Customized SNS Notification via Lambda function using EventBridge

# ==============================================

# Set SNS Topic

In [44]:
def set_sns_topic(sns_client, sns_topic_name):
    topic_response=None
    topic_arns = sns_client.list_topics()['Topics']
    
    if topic_arns != []:
        for topic in topic_arns:
            if sns_topic_name in topic['TopicArn']:
                print("\n Topic name <<{}>> having SNS ARN <<{}>> already exists. \n".format(sns_topic_name, topic['TopicArn']))
                break
        else:
            print(f'\n New topic <<{sns_topic_name}>> created successfully \n')
            topic_response = sns_client.create_topic(
                                Name=sns_topic_name
                            )
    else:
        print(f'\n New topic <<{sns_topic_name}>> created successfully \n')
        topic_response = sns_client.create_topic(
                            Name=sns_topic_name
                        )
        
    return topic_response

In [45]:
set_sns_topic(sns_client, sns_topic_name)


 Topic name <<PipelineExecutionAlarmTopic>> having SNS ARN <<arn:aws:sns:us-east-1:791574662255:PipelineExecutionAlarmTopic>> already exists. 



# Set SNS Subscribers

In [40]:
# subcribe endpoint to sns
def sns_subscribe(sns_client, sns_topic_arn, protocol, endpoint):
    subscribe_response=None
    
    subscription_arns = sns_client.list_subscriptions_by_topic(TopicArn=sns_topic_arn)['Subscriptions']

    if subscription_arns != []:
        for subscription in subscription_arns:
            if ('Protocol' in subscription) and (subscription['Protocol']==protocol) and (subscription['Endpoint']==endpoint):
                print(f"Subscription name <<{subscription['Endpoint']}>> having protocol <<{subscription['Protocol']}>> already exist.")
                break
        else:
            subscribe_response = sns_client.subscribe(
                                            TopicArn= sns_topic_arn,
                                            Protocol=protocol,
                                            Endpoint=endpoint,
                                            ReturnSubscriptionArn=True
                                        )
            print(f'new subscription - <<{endpoint}>> having protocol <<{protocol}>> created successfully')
    else:
        subscribe_response = sns_client.subscribe(
                                                TopicArn= sns_topic_arn,
                                                Protocol=protocol,
                                                Endpoint=endpoint,
                                                ReturnSubscriptionArn=True
                                            )

        print(f'new subscription - {subscription_name} having protocol {protocol} created successfully')
    
    return subscribe_response

# Adding Subscribers

In [ ]:

protocol=['email']
endpoint=['vchitrakathi@dminc.com']

for protocol, endpoint in zip(protocol, endpoint):
    sns_subscribe(sns_client, sns_topic_arn, protocol, endpoint)

# Adding lambda permissions

In [15]:
def add_lambda_permissions(lambda_client, lambda_function_name, src_arn, account_id, action=None, prefix=None):
    try :
        response = lambda_client.add_permission(
                FunctionName=lambda_function_name,
                StatementId=f"{prefix}-{int(time.time())}",
                Action=action,
                Principal= f"{prefix}.amazonaws.com",
                SourceArn=src_arn,
                SourceAccount=account_id
            )
        
    except Exception as e:
        print(f"Error occurred while adding permissions: {e}")
        
    return response

def update_lambda_config(lambda_client, lambda_function_name, src_arn):
    try :
        response = lambda_client.update_function_configuration(
                        FunctionName=lambda_function_name,
                        Environment={
                            'Variables': {
                                'SNS_TOPIC_ARN': src_arn
                            }
                        }
                    )
        
    except Exception as e:
        print(f"Error occurred while updating lambda config: {e}")
        
    return response
        

# SNS Lambda trigger to set rule and put lambda function as target

In [11]:
def sns_lambda_trigger(region, account_id, sns_topic_arn, sns_client, event_bridge_client, lambda_client, lambda_function_name, lambda_arn):
    
    response=None

    # Define the EventBridge rule name and description
    rule_name = 'SNSLambdaEventRuleTrigger'
    rule_description = 'Event to trigger Custom SNS Notification via Lambda function on state change of alarm.'
#     rule_description = 'Trigger sns notification via a Lambda function when a SageMaker pipeline execution is successful and a model is registered'
    
    # Define the Event Pattern
#     event_pattern = {
#           "source": ["aws.sns"],
#           "detail-type": ["AWS API Call via CloudTrail"],
#           "detail": {
#             "eventSource": ["sns.amazonaws.com"]
#           }
#         }
    

    event_pattern = {
                "source": [
                  "aws.cloudwatch"
                ],
                "detail-type": [
                  "CloudWatch Alarm State Change"
                ],
                "detail": {
#                     "alarmName": [
#                         model_alarm_name,
#                         pipeline_alarm_name
#                     ],
                  "state": {
                    "value": [
                      "ALARM", "OK"
                    ]
                  }
                }
              }


#     event_pattern = {
#                   "source": [
#                     "aws.sagemaker"
#                   ],
#                   "detail-type": [
#                     "SageMaker Model Building Pipeline Execution Status Change"
#                   ],
#                   "detail": {
#                     "currentPipelineExecutionStatus": ["Succeeded"]
#                 }
#             }


    # Put the EventBridge rule
    rule_response = event_bridge_client.put_rule(
        Name=rule_name,
        Description=rule_description,
        EventPattern=json.dumps(event_pattern),
        State='ENABLED'
    )
    
    event_rule_arn = rule_response['RuleArn']
    
    # add permissions to Events, SNS
    add_lambda_permissions(lambda_client, lambda_function_name, event_rule_arn, account_id, action='lambda:InvokeFunction', prefix='events')
    add_lambda_permissions(lambda_client, lambda_function_name, sns_topic_arn, account_id, action='lambda:InvokeFunction', prefix='sns')
    update_lambda_config(lambda_client, lambda_function_name, sns_topic_arn)
   
    
    input_msg = {"subject": "Sagemaker Pipeline Execution Results",
                 "message": "Pipeline execution and model registration succeeded"}
    
    # Define the EventBridge target
    lambda_target = {
                'Arn': lambda_arn, # Replace with the actual lambda ARN
                'Id': 'SNSLambdaFunctionTarget',
#                 'Input': json.dumps(input_msg)
            }
    
    sns_target = {
                'Arn': sns_topic_arn, # Replace with the actual topic arn
                'Id': 'SNSFunctionTarget'
            }
    

    # Add the target to the EventBridge rule
    event_bridge_client.put_targets(
        Rule=rule_name,
        Targets=[lambda_target]
    )

    # Wait for the target to become active
    while True:
        response_rule = event_bridge_client.describe_rule(Name=rule_name)
        # response_rule = event_bridge_scheduler.get_schedule(Name=schedule_name)

        if response_rule['State'] == 'ENABLED':
            print('\n === Event Scheduled Successfully ===== \n')
            break
        else:
            print(f'response rule: {response_rule}')
        time.sleep(5)

    # Check the target status
    response = event_bridge_client.list_targets_by_rule(Rule=rule_name)
    
    return response


In [7]:
# response = sns_lambda_trigger(region, account_id, sns_topic_arn, sns_client, event_bridge_client, lambda_client, lambda_function_name, lambda_arn)
# print(response)

# Remove rules

In [ ]:
rule_name = 'SNSLambdaEventRuleTrigger'
event_bridge_client.describe_rule(Name=rule_name)
event_bridge_client.list_targets_by_rule(Rule=rule_name)

In [14]:
rule_name = 'SNSLambdaEventRuleTrigger'
lambda_target_id = 'SNSLambdaFunctionTarget'
sns_target_id='SNSFunctionTarget'

target_ids=[lambda_target_id, sns_target_id]

def remove_event_rule(event_bridge_client, rule_name, target_ids):

    # Disable the EventBridge rule
    event_bridge_client.disable_rule(Name=rule_name)

    # Remove the target for the EventBridge rule
    event_bridge_client.remove_targets(Rule=rule_name, Ids=target_ids)
    
    # delete rule
    event_bridge_client.delete_rule(Name=rule_name)

# =============================================================================
# remove_event_rule(event_bridge_client, rule_name, target_ids)

# ================================================

# Steps related to lambda function and role creation

In [8]:
from pipeline_utils import *
from zipfile import ZipFile

In [ ]:
def add_iam_permissions(name):
    print("Adding permissions to AWS Lambda function's IAM role ...")
    add_execution_role = iam_client.attach_role_policy(
            RoleName=name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
    
    add_execution_role = iam_client.attach_role_policy(
            RoleName=name,
            PolicyArn='arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'
        )
    
    add_execution_role = iam_client.attach_role_policy(
            RoleName=name,
            PolicyArn='arn:aws:iam::aws:policy/AmazonSNSFullAccess'
        )
    
#     add_execution_role = iam_client.attach_role_policy(
#             RoleName=name,
#             PolicyArn='arn:aws:iam::aws:policy/CloudWatchFullAccess'
#         )
    
#     add_execution_role = iam_client.attach_role_policy(
#                 RoleName=name,
#                 PolicyArn=sns_policy_response['Policy']['Arn']
#             )
    
    print("SUCCESS: Successfully added permissions AWS Lambda function's IAM role!")

def create_lambda_role(iam_client, role_name, fcn_name, topic_arn, region, account_id):
    iam_desc = 'IAM Policy for Lambda triggering SNS Notification'
    fcn_desc = 'AWS Lambda function for automatically triggering SNS notification'

    # Define IAM Trust Policy for Lambda's role
    iam_trust_policy = {
    'Version': '2012-10-17',
    'Statement': [
                  {
                    'Effect': 'Allow',
                    'Principal': {
                      'Service': 'lambda.amazonaws.com'
                    },
                    'Action': 'sts:AssumeRole'
                  }
    ]
    }

    # Create the IAM policy
    sns_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "logs:CreateLogGroup",
                "Resource": f"arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{fcn_name}"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogStream",
                    "logs:PutLogEvents"
                ],
                "Resource": [
                    f"arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{fcn_name}:*"
                ]
            },
            {
                "Effect": "Allow",
                "Action": "sns:Publish",
                "Resource": topic_arn
            }
        ]
    }

    print('Creating an IAM role for AWS Lambda function ...')
    create_iam_role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(iam_trust_policy),
        Description=iam_desc
        )
    
    print('SUCCESS: Successfully created IAM role for AWS Lambda function!')
    
    # Attach the policy to the Lambda function's execution role
#     sns_policy_response = iam_client.create_policy(
#         PolicyName='lambda-sns-trigger-policy',
#         PolicyDocument=json.dumps(sns_policy)
#     )

#     iam_client.attach_role_policy(
#                     RoleName=role_name,
#                     PolicyArn=sns_policy_response['Policy']['Arn']
#                 )


    time.sleep(10)
    
    add_iam_permissions(role_name)

    return {
            'arn': create_iam_role['Role']['Arn'],
            'name': create_iam_role['Role']['RoleName']
        }

In [9]:
# Create IAM role for Lambda function

role_name = "sns-lambda-evt-trigger-role" #-{time.strftime('%d-%H-%M-%S', time.gmtime())}"
# fcn_name = "sns-lambda-evt-trigger-fcn" #-{time.strftime('%d-%H-%M-%S', time.gmtime())}"
topic_arn = 'arn:aws:sns:us-east-1:791574662255:PipelineExecutionAlarmTopic'

#Create IAM role for the Lambda function
# lambda_role = create_lambda_role(iam_client, role_name, lambda_function_name, topic_arn, region, account_id)

In [10]:
lambda_role = {'arn': 'arn:aws:iam::791574662255:role/sns-lambda-evt-trigger-role'}
print(lambda_role)

{'arn': 'arn:aws:iam::791574662255:role/sns-lambda-evt-trigger-role'}
{'arn': 'arn:aws:iam::791574662255:role/sns-lambda-evt-trigger-role'}


In [11]:
#Zip AWS Lambda function code
#Write code to a .py file
# with open('lambda_function.py', 'w') as f:
#     f.write(inspect.cleandoc(lambda_code))


lambda_output_path = 'lambda_output'
module_name='sns_trigger_lambda_function'

os.makedirs(name=lambda_output_path, exist_ok=True)

zip_path = os.path.join(lambda_output_path, 'function.zip')

#Compress file into a zip
with ZipFile(zip_path,'w') as z:
    z.write('sns_trigger_lambda_function.py')

#Use zipped code as AWS Lambda function code
with open(zip_path, 'rb') as f:
    fcn_code = f.read()

shutil.rmtree(lambda_output_path)

#Create AWS Lambda function
lambda_arn = create_lambda(module_name, lambda_function_name, fcn_code, lambda_role['arn'])

Creating AWS Lambda function ...
Creating AWS Lambda function ...
SUCCESS: Successfully created AWS Lambda function!
SUCCESS: Successfully created AWS Lambda function!


In [12]:
print(lambda_arn)

arn:aws:lambda:us-east-1:791574662255:function:sns-lambda-evt-trigger-fcn
arn:aws:lambda:us-east-1:791574662255:function:sns-lambda-evt-trigger-fcn


In [46]:
# This file contains a small sample of ovr data
sagemaker.s3.S3Uploader.upload("./data/ovr_content_data_v3.csv", 
                               f"s3://{bucket}/{train_data_dir_prefix}")
#wait for file to finish uploading 
time.sleep(5)